<a href="https://colab.research.google.com/github/JevinKOpenEyes/colab-copies/blob/main/Tagging_and_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install -q auto-gptq transformers tika aspose-words accelerate

In [82]:
import requests
from tika import parser
from bs4 import BeautifulSoup
from io import StringIO

In [83]:
# This line gets ssf6.pdf files metadata from the google drive URI
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/66acaf7a1571ea56e405e9c2")
resp = request.json()

# Getting file metadata
file_path = (resp["data"]["file_path"])
file_name = (resp["data"]["file_name"])
_id = (resp["data"]["_id"])
file_type = (resp["data"]["file_type"])
user_file_name = (resp["data"]["file_name"])
type_of_question = (resp["data"]["type_of_question"])

In [84]:
if file_type == "pdf" or file_type == "txt":
    Text_pdf = []
    # Read PDF file
    data = parser.from_file(file_path, xmlContent=True)
    xhtml_data = BeautifulSoup(data['content'])
    for i, content in enumerate(xhtml_data.find_all('div', attrs={'class': 'page'})):
        # Parse PDF data using TIKA (xml/html)
        # It's faster and safer to create a new buffer than truncating it
        # https://stackoverflow.com/questions/4330812/how-do-i-clear-a-stringio-object
        _buffer = StringIO()
        _buffer.write(str(content))
        parsed_content = parser.from_buffer(_buffer.getvalue())

        # Add pages
        if parsed_content['content']!=None:
            text = parsed_content['content'].strip()
            Text_pdf.append(text)

2024-08-05 08:09:35,211 [MainThread  ] [INFO ]  Retrieving https://drive.google.com/u/0/uc?id=1ozCc6YVeBOy5047AeIJpj1WvbOJuuZDr&export=download to /tmp/u-0-uc.
INFO:tika.tika:Retrieving https://drive.google.com/u/0/uc?id=1ozCc6YVeBOy5047AeIJpj1WvbOJuuZDr&export=download to /tmp/u-0-uc.


In [85]:
Text_pdf

['HB 207  2024 \n\n\n\n\n\n\n\n\n\n\n\nCODING: Words stricken are deletions; words underlined are additions. \n\n\nhb0207-00 \n\n\nPage 1 of 4 \n\n\nF L O R I D A  H O U S E  O F  R E P R E S E N T A T I V E S \n\n\n\n\n\n\n\n\n\n\n\nA bill to be entitled 1 \n\n\nAn act relating to social media protection for minors; 2 \n\n\ncreating s. 501.174, F.S.; providing a definition; 3 \n\n\nrequiring social media platforms to disclose specified 4 \n\n\npolicies and provide specified resources, measures, 5 \n\n\nand disclaimers; authorizing social media platforms to 6 \n\n\npost specified compliance statements on their Internet 7 \n\n\nhomepage or platform user login page; prohibiting 8 \n\n\ncertain schools from using or having an account on 9 \n\n\ncertain social media platforms and from requiring 10 \n\n\nstudents to register, enroll, or participate in social 11 \n\n\nmedia platforms for educational purposes; providing 12 \n\n\napplicability; providing an effective date. 13 \n\n\n 14 \n\n\nB

In [86]:
combined_text = "\n".join(Text_pdf)

In [87]:
combined_text

'HB 207  2024 \n\n\n\n\n\n\n\n\n\n\n\nCODING: Words stricken are deletions; words underlined are additions. \n\n\nhb0207-00 \n\n\nPage 1 of 4 \n\n\nF L O R I D A  H O U S E  O F  R E P R E S E N T A T I V E S \n\n\n\n\n\n\n\n\n\n\n\nA bill to be entitled 1 \n\n\nAn act relating to social media protection for minors; 2 \n\n\ncreating s. 501.174, F.S.; providing a definition; 3 \n\n\nrequiring social media platforms to disclose specified 4 \n\n\npolicies and provide specified resources, measures, 5 \n\n\nand disclaimers; authorizing social media platforms to 6 \n\n\npost specified compliance statements on their Internet 7 \n\n\nhomepage or platform user login page; prohibiting 8 \n\n\ncertain schools from using or having an account on 9 \n\n\ncertain social media platforms and from requiring 10 \n\n\nstudents to register, enroll, or participate in social 11 \n\n\nmedia platforms for educational purposes; providing 12 \n\n\napplicability; providing an effective date. 13 \n\n\n 14 \n\n\nBe

In [88]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [89]:
facebook_model_summary = summarizer(Text_pdf, max_length=300, min_length=100, do_sample=False)

In [90]:
merged_summary_facebook = " ".join(item['summary_text'] for item in facebook_model_summary)
print(merged_summary_facebook)

 HB 207 is a bill to be entitled "An act relating to social media protection for minors" It would require social media platforms to disclose specified policies and provide specified resources, measures, and disclaimers. It would also prohibit certain schools from using or having an account on social media sites. The bill was passed by the Florida Legislature and signed into law by Gov. Rick Scott on January 1, 2014. It is expected to become law by the end of this year or early next year. It will be the first of its kind in the United States. Beginning January 1, 2025, a social media platform must disclose the following policies. It must be clearly, concisely, prominently, and clearly written using language suited to the age of users. The platform must consider the best interests of platform users who are under 18 when designing, developing, and providing services. It should be clearly labeled, conspicuous, and readily accessible through a readily accessible link on a homepage or login 

In [91]:
merged_summary_facebook

' HB 207 is a bill to be entitled "An act relating to social media protection for minors" It would require social media platforms to disclose specified policies and provide specified resources, measures, and disclaimers. It would also prohibit certain schools from using or having an account on social media sites. The bill was passed by the Florida Legislature and signed into law by Gov. Rick Scott on January 1, 2014. It is expected to become law by the end of this year or early next year. It will be the first of its kind in the United States. Beginning January 1, 2025, a social media platform must disclose the following policies. It must be clearly, concisely, prominently, and clearly written using language suited to the age of users. The platform must consider the best interests of platform users who are under 18 when designing, developing, and providing services. It should be clearly labeled, conspicuous, and readily accessible through a readily accessible link on a homepage or login

In [92]:
from transformers import pipeline

pipe = pipeline(model="facebook/bart-large-mnli")


In [93]:
classified_tag_facebook = pipe(merged_summary_facebook,
    candidate_labels=["Budget", "Education", "Government", "Data", "State Response","Healthcare","Other", "Privacy", "Military",  "Telecom", "Energy","Transportation","Social", "Labor","AG"],
)

In [94]:
classified_tag_facebook

{'sequence': ' HB 207 is a bill to be entitled "An act relating to social media protection for minors" It would require social media platforms to disclose specified policies and provide specified resources, measures, and disclaimers. It would also prohibit certain schools from using or having an account on social media sites. The bill was passed by the Florida Legislature and signed into law by Gov. Rick Scott on January 1, 2014. It is expected to become law by the end of this year or early next year. It will be the first of its kind in the United States. Beginning January 1, 2025, a social media platform must disclose the following policies. It must be clearly, concisely, prominently, and clearly written using language suited to the age of users. The platform must consider the best interests of platform users who are under 18 when designing, developing, and providing services. It should be clearly labeled, conspicuous, and readily accessible through a readily accessible link on a home

In [95]:
Text_pdf

['HB 207  2024 \n\n\n\n\n\n\n\n\n\n\n\nCODING: Words stricken are deletions; words underlined are additions. \n\n\nhb0207-00 \n\n\nPage 1 of 4 \n\n\nF L O R I D A  H O U S E  O F  R E P R E S E N T A T I V E S \n\n\n\n\n\n\n\n\n\n\n\nA bill to be entitled 1 \n\n\nAn act relating to social media protection for minors; 2 \n\n\ncreating s. 501.174, F.S.; providing a definition; 3 \n\n\nrequiring social media platforms to disclose specified 4 \n\n\npolicies and provide specified resources, measures, 5 \n\n\nand disclaimers; authorizing social media platforms to 6 \n\n\npost specified compliance statements on their Internet 7 \n\n\nhomepage or platform user login page; prohibiting 8 \n\n\ncertain schools from using or having an account on 9 \n\n\ncertain social media platforms and from requiring 10 \n\n\nstudents to register, enroll, or participate in social 11 \n\n\nmedia platforms for educational purposes; providing 12 \n\n\napplicability; providing an effective date. 13 \n\n\n 14 \n\n\nB

In [96]:
from transformers import pipeline
pipe = pipeline("summarization", model="google/pegasus-xsum")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [97]:
google_model_summary = pipe(Text_pdf, max_length=300, min_length=100, do_sample=False)

Your max_length is set to 300, but your input_length is only 295. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=147)


In [98]:
merged_summary_google = " ".join(item['summary_text'] for item in google_model_summary)

In [99]:
merged_summary_google = """HB 207 is a bill to be entitled "An act relating to social media protection for minors" It would require social media platforms to disclose specified policies and provide specified resources, measures, and disclaimers. It would also prohibit schools from using or having an account on social media. Beginning January 1, 2025, a social media platform must disclose the following policies. It must be clearly, concisely, prominently, and clearly written using language suited to the age of users. uses to protect platform users who are under the age of 18. Whether the social media platform collects or sells personal information of platform users. Zip code-based references to local resources for law. This application may be harmful to your mental health. A social media platform that complies with subsections 84, 85, and (3) may post a statement confirming such compliance on 85 its Internet homepage or platform user login page. A public K-12 school may not: (a)  Use or have an account on a social mediaplatform that 88, 89, 90 does not have a statement pursuant to subsection """

In [100]:
from transformers import pipeline

pipe = pipeline(model="facebook/bart-large-mnli")
classfied_tag_google = pipe(merged_summary_google,
    candidate_labels=["Budget", "Education", "Government", "Data", "State Response","Healthcare","Other", "Privacy", "Military",  "Telecom", "Energy","Transportation","Social", "Labor","AG"],
)

In [101]:
classfied_tag_google

{'sequence': 'HB 207 is a bill to be entitled "An act relating to social media protection for minors" It would require social media platforms to disclose specified policies and provide specified resources, measures, and disclaimers. It would also prohibit schools from using or having an account on social media. Beginning January 1, 2025, a social media platform must disclose the following policies. It must be clearly, concisely, prominently, and clearly written using language suited to the age of users. uses to protect platform users who are under the age of 18. Whether the social media platform collects or sells personal information of platform users. Zip code-based references to local resources for law. This application may be harmful to your mental health. A social media platform that complies with subsections 84, 85, and (3) may post a statement confirming such compliance on 85 its Internet homepage or platform user login page. A public K-12 school may not: (a)  Use or have an acco

In [102]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [103]:
sequence_to_classify = "Bharti Airtel, the second largest operator, provided salary hikes in the range of 10-12%. Loss-making Vodafone Idea, with over 9,000 employees, also gave hikes of 8-11%. Reliance Jio Infocomm has yet to announce the hikes for this year."

In [104]:

candidate_labels = ["Budget", "Education", "Government", "Data", "State Response","Healthcare","Other", "Privacy", "Military",  "Telecom", "Energy","Transportation","Social", "Labor","AG"]
classifier(sequence_to_classify, candidate_labels)


{'sequence': 'Bharti Airtel, the second largest operator, provided salary hikes in the range of 10-12%. Loss-making Vodafone Idea, with over 9,000 employees, also gave hikes of 8-11%. Reliance Jio Infocomm has yet to announce the hikes for this year.',
 'labels': ['Telecom',
  'Social',
  'Other',
  'AG',
  'Privacy',
  'Data',
  'Budget',
  'Labor',
  'State Response',
  'Energy',
  'Government',
  'Military',
  'Transportation',
  'Healthcare',
  'Education'],
 'scores': [0.27929526567459106,
  0.1034245565533638,
  0.1009020283818245,
  0.08975376188755035,
  0.07511309534311295,
  0.0498567558825016,
  0.04803336784243584,
  0.047973550856113434,
  0.03690702095627785,
  0.03668290749192238,
  0.03582572937011719,
  0.03137030825018883,
  0.02977333404123783,
  0.019053194671869278,
  0.016035133972764015]}